# Выбор локации для скважины

Допустим, вы работаете в добывающей компании «ГлавРосГосНефть». Нужно решить, где бурить новую скважину.

Вам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Постройте модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. Проанализируйте возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

In [1]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from numpy.random import RandomState
from scipy import stats as st

## Загрузка и подготовка данных

In [2]:
data_geo1 = pd.read_csv('/datasets/geo_data_0.csv')
data_geo2 = pd.read_csv('/datasets/geo_data_1.csv')
data_geo3 = pd.read_csv('/datasets/geo_data_2.csv')

In [3]:
data_geo1.info()
data_geo2.info()
data_geo3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   id       100000 non-null  object 
 1   f0       100000 non-null  float64
 2   f1       100000 non-null  float64
 3   f2       100000 non-null  float64
 4   product  100000 non-null  float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
 #   Column   Non-Null 

In [5]:
#признаки для обучения моделей
features_geo1 = data_geo1.drop(['id','product'], axis=1)
features_geo2 = data_geo2.drop(['id','product'], axis=1)
features_geo3 = data_geo3.drop(['id','product'], axis=1)

#целевые признаки для обучения модели
target_geo1 = data_geo1['product']
target_geo2 = data_geo2['product']
target_geo3 = data_geo3['product']


<div>
    <b> Мой комментарий 🙃:</b>
    <br> Порядок действий : загрузил данные --> разделил данные на таргеты и фичи (только айди убрал из данных)
</div>

## Обучение и проверка модели

In [6]:
# разбиваю выборки на тестовую и валидициоонную для каждого датасета в соотношеннии 3 : 1
features_geo1_train , features_geo1_valid, target_geo1_train, target_geo1_valid = train_test_split(features_geo1, target_geo1, test_size=.25, random_state=145)

features_geo2_train , features_geo2_valid, target_geo2_train, target_geo2_valid = train_test_split(features_geo2, target_geo2, test_size=.25, random_state=145)

features_geo3_train , features_geo3_valid, target_geo3_train, target_geo3_valid = train_test_split(features_geo3, target_geo3, test_size=.25, random_state=145)

In [7]:
#сброшу индексы у таргета чтобы потом можно было использовать для поиска целевых значений предсказания с помощью iloc
target_geo1_valid = target_geo1_valid.reset_index(drop=True)
target_geo2_valid = target_geo2_valid.reset_index(drop=True)
target_geo3_valid = target_geo3_valid.reset_index(drop=True)

<div>
    <b> Мой комментарий 🙃:</b>
    <br> Разбил данные каждого региона на признаки и целевой признак
    <br> Сбросил индексы у target_geo_valid
</div>

In [8]:
#напишу метод который будет обучать модель линейной регрессии и возвращать predictions модели , также будет выводить средний запас предсказанного сырья и RMSE модели

def fit_predict(X_train, y_train, X_valid, y_valid):
    model = LinearRegression()
    model.fit(X_train, y_train)
    predictions = pd.Series(model.predict(X_valid))
    
    print(f"Модель : средний запас сырья {predictions.mean()} , RMSE модели {mean_squared_error(y_valid, predictions) ** 0.5}")
    return predictions

In [9]:
predictions_model_1 = fit_predict(features_geo1_train, target_geo1_train, features_geo1_valid, target_geo1_valid)

Модель : средний запас сырья 92.49714612578681 , RMSE модели 37.67377658924483


In [10]:
predictions_model_2 = fit_predict(features_geo2_train, target_geo2_train, features_geo2_valid, target_geo2_valid)

Модель : средний запас сырья 69.02412249554622 , RMSE модели 0.8958566393446236


In [11]:
predictions_model_3 = fit_predict(features_geo3_train, target_geo3_train, features_geo3_valid, target_geo3_valid)

Модель : средний запас сырья 94.99445800590067 , RMSE модели 40.072072843232085


<div>
    <b> Мой комментарий 🙃:</b>
    <br> linear_model_1 и linear_model_3 показывают очень низкий RMSE на валидационных выборках - скорее всего недообучение , а вот linear_model_2 показывает очень высокий RMSE на валидационной выборке , скорее всего данные очень хорошо описываются линейно или переобучение, 
</div>

## Подготовка к расчёту прибыли

In [12]:
SAMPLES = 500 #500 точек для разведки региона 
BEST_SAMPLES = 200 # 200 точек лучших после разведки
DEV_BUDGET = 10000000000 # бюджет на разработку скважин в регионе
UNIT_INCOME = 450000 # доход с еденицы продукта

BREAK_DEV_VOLUME = DEV_BUDGET/(BEST_SAMPLES*UNIT_INCOME)

print(f'Объем продукта для безубыточной разработки скважин , должен быть больше чем {round(BREAK_DEV_VOLUME,2)} тыс барралей в скважине ')
# с учетом того что выбраны для разработки будут всего 200 лучших скважин

Объем продукта для безубыточной разработки скважин , должен быть больше чем 111.11 тыс барралей в скважине 


In [13]:
for i,j in zip([predictions_model_1, predictions_model_2, predictions_model_3], range(1,4)):
    print(f'Средний запас предсказанного сырья в {j} регионе {i.mean()} тыс. барралей')


Средний запас предсказанного сырья в 1 регионе 92.49714612578681 тыс. барралей
Средний запас предсказанного сырья в 2 регионе 69.02412249554622 тыс. барралей
Средний запас предсказанного сырья в 3 регионе 94.99445800590067 тыс. барралей


<div>
    <b> Мой комментарий 🙃:</b>
    <br> По расчетам при исследовании 200 скважин необходимый объем продукта для безубыточной разработки скважин в регионе должен быть больше чем 111 тыс барралей в скважине , средний объем предсказанного объема сырья по каждому региону  не превосходит этот объем  , так что разрабатывать нужно  регионы с средним показателем близким к этому значению т.е geo1 и geo3
</div>

## Расчёт прибыли и рисков 

In [14]:
# функция для расчета прибыли ищем 200 лучших
def profit_calc(target_value, predictions): # нахожу 200 скважин с макс значением предсказанного запаса сырья ---> cуммирую целевое значение по этим 200 лучших скважин умножаю на стоимость еденицы продукта вычитаю бюджет ===<> возвращаю прибыль по 200 лучшим скважинам 
    predictions = predictions.sort_values(ascending=False)# sorted
    predictions = predictions.iloc[:200]#take 200 best
    target_value = target_value.loc[predictions.index]# find target
    return round(target_value.sum() * UNIT_INCOME - DEV_BUDGET)#find profit

<div>
    <b> Мой комментарий 🙃:</b>
    <br> Написал метод для расчета 200 лучших скважин по объему сырья , расчитал доход по трем геоданным
</div>

In [15]:
# метод для нахождения 1000 бутстрап выборок для поиска распределения прибыли  
def bootstrap(target, predictions):
    state = RandomState(27)
    array = []
    for i in range(1000):
        predictions_sampled = predictions.sample(n=500, replace=True, random_state=state)
        array.append(profit_calc(target, predictions_sampled))
    return pd.Series(array)

In [20]:
def mean_and_interval(target, predictions):
    array = bootstrap(target, predictions)
    print(f'средняя прибыль {array.mean()}')
    print(f'95% выручки будет находиться в интервале от {array.quantile(0.025)} и до {array.quantile(0.975)}')
    print(f'вероятность убытков : {round((array.values<0).mean() * 100,1)}%')

In [22]:
array = bootstrap(target_geo1_valid, predictions_model_1)
print((array.values < 0).mean())

0.043


In [23]:
mean_and_interval(target_geo1_valid, predictions_model_1)

средняя прибыль 414571755.603
95% выручки будет находиться в интервале от -39240041.64999999 и до 862590378.4749998
вероятность убытков : 4.3%


In [24]:
mean_and_interval(target_geo2_valid, predictions_model_2)

средняя прибыль 469962131.851
95% выручки будет находиться в интервале от 66588936.37500001 и до 863804197.7249999
вероятность убытков : 1.1%


In [25]:
mean_and_interval(target_geo3_valid, predictions_model_3)

средняя прибыль 332228218.322
95% выручки будет находиться в интервале от -190128540.67499998 и до 845111365.15
вероятность убытков : 10.6%


<div>
    <b> Мой комментарий 🙃:</b>
    <br> Нашел по данным предсказаний трех моделей в каждом регионе , с помощью бутстрап выборок , доверительный интервал средней прибыли , среднюю прибыль  , а также риск убытков
    <br>По этим данным можно сделать вывод , что в регионах 1  и 3 cредняя прогнозируемая прибыль меньше чем во втором регионе при том что вероятность рисков больше 
    <br>Мои рекомендации производить разработку скважин в 2 регионе , потому что средняя потенциальная прибыль в этом регионе больше , при том что риск убытков меньше чем в других регионах.
</div>